# Multi Agent Collaboration and Competition Project Report

#### Train 2 Reinforcement Learning Agents to play Tennis

  ## Introduction

#### Project Overview

For this project, two agents in an environment control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1. If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01. Thus, the goal of each agent is to keep the ball in play.

#### Environment
![Unity ML-Agents Tennis Environment](./images/tennis.png)

For this project, I worked with an environment similar but not identical to the [Tennis](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Learning-Environment-Examples.md#tennis) environment on the Unity ML-Agents Github page.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Each agent receives its own, local observation. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping.

The task is episodic, and in order to solve the environment, your agents must get an average score of +0.5 (over 100 consecutive episodes, after taking the maximum over both agents). Specifically,

   - After each episode, I added up the rewards that each agent received (without discounting), to get a score for each agent. This yields 2 (potentially different) scores. Then the maximum of these 2 scores was taken.
   - This yields a single score for each episode.

The environment is considered solved, when the average (over 100 episodes) of those scores is at least +0.5.

## Methodology and Algorithm

### Architecture

Deep DPG (DDPG) can learn competitive policies for different tasks using low-dimensional observations (e.g. cartesian coordinates or joint angles). 
Since Value based approaches are not suited for this kind of tasks, I used an extention of Deep Deterministic Policy Gradient (DDPG) algorithm called MADDPG. Multi-Agent Deep Deterministic Policy Gradient (MADDPG) was originially stated in _[Multi-Agent Actor-Critic for Mixed Cooperative-Competitive Environments](https://arxiv.org/abs/1706.02275)_.


As presented in the paper, Multi-agent policy gradient algorithm (MADDPG) agents learn a centralized critic based on the observations and actions of all agents.

This is a simple extension of actor-critic policy gradient methods where the critic is augmented with extra information about the policies of other agents, while the actor only has access to local information. After training is completed, only the local actors are used at execution phase, acting in a decentralized manner and equally applicable in cooperative and competitive settings.


![Multi Agent Actor Critic](./images/actor_critic.png)
> Figure 1: Multi-agent decentralized actor with centralized critic ([Lowe and Wu et al](https://arxiv.org/abs/1706.02275))


### Code Structure
There is just one IPython notebook that contains all my implementations. In the notebook defined several helper functions and classes. Which include the ones discussed below. For the implementation, Pytorch and the environment provided by Udacity was used.

> ```Model```: In this code cell I implemented the model, Actor and the Critic class using PyTorch framework. This Method - Actor-Critic method combines both Value-based and Policy-based methods. What makes this implementation unique is the decentralized actor with centralized critic approach.
- ```class Actor```
    - An Input Layer: Which the value depends on the state_size parameter.
    - Two other fully connected layer with in_units=200 and out_units=150.
    - An output layer: The value of which depends on the action_size parameter.
    - ```.reset_parameters()```: This methods helps to initialize the weights using uniform distribution.
    - ```.forward()```: method maps states to corresponding actions. A non-linear function called ReLu activation function was used for the hidden layers and tanh was used for the output layer to maintain values between -1 and 1.  
- ```class Critic```
    - Input Layer: The size depends on the state_size parameter.
    - Two (2) fully connected linear layer
    - Ouput layer: this layer gives a single value.
    - ```.reset_parameters()```: This methods helps to initialize the weights using uniform distribution.
    - ```.forward()```: this method implements the forward pass and maps (state action) pair. ReLu activation function was used for the hidden layers. 

> - ```Class OUNoise```: In this method **[Ornstein-Uhlenbeck Process]**(https://arxiv.org/pdf/1509.02971.pdf) was implemented. This process adds a certain amount of noise to the action values at each timestep and help us address the trade-off between Exploitation Vs. Exploration Dilema. This was originally implemented in CONTINUOUS CONTROL WITH DEEP REINFORCEMENT LEARNING paper. This allows the agent to maintain velocity and explore the action space with more continuity.
    - parameters (mu, theta, sigma, seed) were initialized.
    - ```.reset()```: It create a copy of the internal state  with parameter, mu.
    - ```.sample()```: This update the internal state and return it as a noise sample using theta and sigma parameters.

> - ```Class ReplayBuffer```: In this class, experience replay was implemented, which allows the Agent to learn from past experiences. So this fixed size buffer can store experince tuples. For the 2 agents, the experiences are then utilized by the central critic to enable the agents learn from each others' experiences.
    - The replay buffer parameters and experience tuple were initialized.
    - ```.add()```: The method adds new Experience tuple _(state, action, reward, next_state, done)_ to the memory
    - ```.sample()```: This samples and return Random batch of experiences from the memory.

> ```The Agent```: This Cell contains the implementation of the Action-Critic logic.
- ```Class Agent```:
    - The local and target networks were initialized separately for both the action and the critic to improve stability.
    - ```.step()```: Learn at given interval.
    - ```.act()```: The method return Actions for a given state based on the current policy. 
    - ```.learn()```: Here, the policy value parameters were updated with selected experiences. The critic network was first implemented, after the forward pass, I calculated the loss and performed the optimiation step.
    - ```.soft_update()```: The model parameters were updated here.

> ```MADDPG Class ```: This class contains different methods responsible for training and saving the best model

> ```Training the MDDPG Agent```: This is where the function responsible agennt - environment interaction and training the model is defined.

### Hyperparameters
These are hyperparameters used to train the agent. It was difficult getting these hyperparameters because most of the previous ones doesn't converge at a reasonable episode.

```
BUFFER_SIZE  = int(1e5)  
BATCH_SIZE   = 250       
GAMMA        = 0.99      
TAU          = 1e-3      
LR_ACTOR     = 1e-4      
LR_CRITIC    = 1e-3      
WEIGHT_DECAY = 0   
OU_THETA     = 0.15
OU_SIGMA     = 0.2

```

For the Actor and Critic Network ```Adam Optimizer``` 

## Result

After training the Agents with the specified hyperparamters and architecture, the plot below was generated. The plot shows the performance of the agents over several episodes.

![Result.png](./images/result.png)

![Chart.png](./images/chart.png)

## Ideas on Performance Improvement
In the future, I will consider improvement on this project using:

- **Batch Normalization**: Batch normalization addresses this problem by scaling the features to be within the same range throughout the model and across different environments and units. I used it within the network in the previous project and it proved tremendeous result. 


- **Priotized Experience Replay**: Experience replay lets online reinforcement learning agents remember and reuse experiences from the past. In prior work, experience transitions were uniformly sampled from a replay memory. However, this approach simply replays transitions at the same frequency that they were originally experienced, regardless of their significance. [Paper](https://arxiv.org/abs/1511.05952)


- **Proximal Policy Optimization**: This technique modifies the parameters of the network in such a way that the new set of parameters is looked for in the immediate neighbourhood of the parameters in the previous iteration of the training. [paper](https://arxiv.org/abs/1707.06347)
